In [1]:
import os
import pymssql

import pandas as pd
import numpy as np
import locale

from dotenv import load_dotenv
from utils_ccd import get_connection
from docxtpl import DocxTemplate


load_dotenv()
conn = get_connection()

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from num2words import num2words

from docx2pdf import convert

/home/eduardo/Dev/ccd/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import subprocess

def generate_pdf(doc_path, path):

    subprocess.call(['soffice',
                  '--headless',
                 '--convert-to',
                 'pdf',
                 '--outdir',
                 path,
                 doc_path])
    return doc_path

In [3]:
descontos_sql = ''' 

SELECT 
    pro.numero_processo,
    pro.ano_processo,
    pro.assunto,
    pm.Descricao as descricao,
    CONCAT(gp.Nome, ' (CPF: ', gp.Documento, ')') as nome,
    processo.dbo.fn_Exe_RetornaValorAtualizado(ed.IdDebito) as valor_multa,
    orgao_info.nome_orgao,
    ed.CodigoStatusDivida as status_divida

FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
    ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm 
    ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed 
    ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
    ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp 
    ON gp.IdPessoa = edp.IDPessoa
OUTER APPLY (
    SELECT TOP 1 vspfr.nome_orgao
    FROM BdDIP.dbo.vwSiaiPessoalFolhaResumida vspfr
    WHERE vspfr.cpf = gp.Documento COLLATE SQL_Latin1_General_CP1_CI_AS
    ORDER BY vspfr.ano DESC, vspfr.mes DESC
) AS orgao_info
WHERE 
    pro.setor_atual = 'CCD'
    AND pm.Descricao = 'DESCONTO EM FOLHA'
    AND ed.CodigoTipoDebito NOT IN (1, 3)

'''
processos_descontos = pd.read_sql(descontos_sql, conn)

/tmp/ipykernel_268114/1758313218.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  processos_descontos = pd.read_sql(descontos_sql, conn)


In [4]:
processos_descontos

,numero_processo,ano_processo,assunto,descricao,nome,valor_multa,nome_orgao,status_divida
0,011032,2016,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,CARLOS JOSÉ FERNANDES (CPF: 76816591453),225.1209,PREFEITURA MUNICIPAL DE MAJOR SALES,1
1,003636,2017,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,JOÃO DIMAS BEZERRA (CPF: 02600404449),15538.9192,None,3
2,003636,2017,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,FRANCISCO DE ASSIS DE SOUZA (CPF: 50288032420),35388.9372,PREFEITURA MUNICIPAL DE JARDIM DE ANGICOS,1
3,003636,2017,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Francisco Nobre Neto (CPF: 77801172434),55867.6200,CÂMARA MUNICIPAL DE JARDIM DE ANGICOS,3
4,003636,2017,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Francisco Nobre Neto (CPF: 77801172434),46556.3774,CÂMARA MUNICIPAL DE JARDIM DE ANGICOS,13
...,...,...,...,...,...,...,...,...
80,000852,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Alexandre Magno de Montenegro Miranda (CPF: 15...,4333.2588,COMPANHIA DE SERVIÇOS URBANOS DE NATAL,1
81,000852,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Maria Solange Ferreira da Silva (CPF: 40632890...,4299.5486,None,3
82,000852,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Jonny Araujo da Costa (CPF: 20119178400),1262.2736,COMPANHIA DE PROCESSAMENTO DE DADOS DO RN,2
83,000852,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Maria Solange Ferreira da Silva (CPF: 40632890...,3846.9782,None,13


In [7]:
def criar_valor_extenso(x):
        if x and x > 1:
            f = lambda c: num2words(int(c), lang='pt_BR')
            return ' reais e '.join([f(y) for y in str('{:.2f}'.format(x)).split('.')]) + ' centavos'
        else:
            return ''

def converter_valor(x):
    if x:
        return locale.currency(x, grouping=True) + ' (' + criar_valor_extenso(x) + ')'
    else:
        return 'R$ 0,00 (zero)'

In [10]:
processos_descontos.fillna(0, inplace=True)

In [11]:
processos_descontos['processo'] = processos_descontos['numero_processo'] + '/' + processos_descontos['ano_processo'].astype(str)
processos_descontos['valor'] = processos_descontos['valor_multa'].apply(converter_valor)  

In [12]:
descontos_aberto = processos_descontos[processos_descontos.status_divida == 1]

In [13]:
descontos_aberto

,numero_processo,ano_processo,assunto,descricao,nome,valor_multa,nome_orgao,status_divida,processo,valor
0,011032,2016,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,CARLOS JOSÉ FERNANDES (CPF: 76816591453),2.251209e+02,PREFEITURA MUNICIPAL DE MAJOR SALES,1,011032/2016,"R$ 225,12 (duzentos e vinte e cinco reais e do..."
2,003636,2017,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,FRANCISCO DE ASSIS DE SOUZA (CPF: 50288032420),3.538894e+04,PREFEITURA MUNICIPAL DE JARDIM DE ANGICOS,1,003636/2017,"R$ 35.388,94 (trinta e cinco mil, trezentos e ..."
14,003636,2017,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Francisco Nobre Neto (CPF: 77801172434),4.102813e+04,CÂMARA MUNICIPAL DE JARDIM DE ANGICOS,1,003636/2017,"R$ 41.028,13 (quarenta e um mil e vinte e oito..."
30,003636,2017,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,JOÃO DIMAS BEZERRA (CPF: 02600404449),4.572236e+03,0,1,003636/2017,"R$ 4.572,24 (quatro mil, quinhentos e setenta ..."
31,002138,2019,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,ABEL BELARMINO DE AMORIM FILHO (CPF: 30732921449),2.162631e+04,PREFEITURA MUNICIPAL DE RAFAEL GODEIRO,1,002138/2019,"R$ 21.626,31 (vinte e um mil, seiscentos e vin..."
32,002138,2019,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,LUDMILA CARLOS AMORIM DE ARAÚJO ROSADO (CPF: 0...,2.162631e+04,PREFEITURA MUNICIPAL DE RAFAEL GODEIRO,1,002138/2019,"R$ 21.626,31 (vinte e um mil, seiscentos e vin..."
40,000088,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,JOSINALDO AMARO DE LIMA (CPF: 73694940472),3.488608e+03,PREFEITURA MUNICIPAL DE SAO TOME,1,000088/2021,"R$ 3.488,61 (três mil, quatrocentos e oitenta ..."
41,001434,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Antônio Carlos Peixoto Nunes (CPF: 00789686406),5.812417e+03,PREFEITURA MUNICIPAL DE PARAÚ,1,001434/2023,"R$ 5.812,42 (cinco mil, oitocentos e doze reai..."
42,001434,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Antônio Carlos Peixoto Nunes (CPF: 00789686406),1.453104e+03,PREFEITURA MUNICIPAL DE PARAÚ,1,001434/2023,"R$ 1.453,10 (mil, quatrocentos e cinquenta e t..."
43,001434,2023,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA,Antônio Carlos Peixoto Nunes (CPF: 00789686406),5.231175e+04,PREFEITURA MUNICIPAL DE PARAÚ,1,001434/2023,"R$ 52.311,75 (cinquenta e dois mil, trezentos ..."


In [24]:
for desconto in descontos_aberto.itertuples():
    doc = DocxTemplate("templates/desconto_folha.docx")
    context = {
        'processo': desconto.processo,
        'assunto': desconto.assunto,
        'nome': desconto.nome,
        'orgao': desconto.nome_orgao,
        'valor': desconto.valor
    }
    nome = desconto.nome.split()[0].lower()
    processo_underline = desconto.processo.replace('/', '_')

    doc.render(context)
    doc.save(f"saidas/desconto_folha/{processo_underline}_{nome}.docx")
    generate_pdf(f"saidas/desconto_folha/{processo_underline}_{nome}.docx", f"saidas/desconto_folha/")
    print(f"Gerado arquivo para o processo {desconto.processo}")

convert /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/011032_2016_carlos.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/011032_2016_carlos.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 011032/2016


convert /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/003636_2017_francisco.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/003636_2017_francisco.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 003636/2017


convert /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/003636_2017_francisco.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/003636_2017_francisco.pdf using filter : writer_pdf_Export
Overwriting: /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/003636_2017_francisco.pdf
Gerado arquivo para o processo 003636/2017


convert /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/003636_2017_joão.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/003636_2017_joão.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 003636/2017


convert /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/002138_2019_abel.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/002138_2019_abel.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 002138/2019


...

convert /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/000062_2025_ronaldo.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/000062_2025_ronaldo.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 000062/2025


convert /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/000065_2025_joão.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/000065_2025_joão.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 000065/2025


convert /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/000067_2025_joão.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/000067_2025_joão.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 000067/2025


convert /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/002013_2025_sergio.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/002013_2025_sergio.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 002013/2025


convert /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/000852_2025_alexandre.docx as a Writer document -> /home/eduardo/Dev/ccd/scripts/saidas/desconto_folha/000852_2025_alexandre.pdf using filter : writer_pdf_Export
Gerado arquivo para o processo 000852/2025
